In [60]:
import os
import copy
import torch
import numpy as np
import config as cfg
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from models.simpleNet import Net
from datamanager import *
from Fed_Algorithms import FedAvg
import sklearn.metrics as metrics
from collections import OrderedDict
from multiprocessing import pool, cpu_count
from multiprocessing import Process
from torch.optim import SGD

class Client():
    def __init__(self, client_id:str, model:nn.Module, data_info:dict=None, device:str=cfg.DEVICE):
        self.id = client_id
        #self.cfg = cfg
        
        self.__model = None
        self.device = device
        
        self.train_info, self.test_info = data_info['train'], data_info['test'] # 함수화하기
        self.trainset, self.testset = FEMNIST(self.train_info), FEMNIST(self.test_info)
        
    @property
    def model(self):             
        return self.__model

    @model.setter
    def model(self, model):
        self.__model = model
    
    def __len__(self):
        return len(self.trainset)
    
    def setup(self):
        self.train_loader = DataLoader(self.trainset, batch_size=16, shuffle=True)
        self.test_loader = DataLoader(self.testset, batch_size=16, shuffle=False)
        self.optimizer = SGD(self.model.parameters(), lr=0.01)     # TODO: utils.get_optimizer(cfg['optim']:str)
        self.criterion = nn.CrossEntropyLoss()                     # TODO: utils.get_loss(cfg['loss']:str)
        self.epochs = 10
    
    def local_train(self)->None:
        self.model.train()
        self.model.to(self.device)
        # TRAINING
        for epoch in range(self.epochs):
            for idx, batch in enumerate(self.train_loader):
                self.optimizer.zero_grad()
                X, Y = batch
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X)
                loss = self.criterion(pred, Y)
                loss.backward()
                self.optimizer.step()
                if "cuda" in self.device : torch.cuda.empty_cache()
        # TESTING
        self.model.eval()
        self.model.to(self.device)
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.train_loader):
                X, Y = batch
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X)
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
                if "cuda" in self.device : torch.cuda.empty_cache()
            train_acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            train_loss = np.average(loss_trace)
            self.model.to('cpu')
        
        print(f'=== Client {self.id} Finished Training {len(self)} samples ===')
        print(f'client:{self.id} | Train Acc:{train_acc*100:.2f} | Train Loss:{train_loss:.4f}')
    
    def local_test(self):
        self.model.eval()
        self.model.to(self.device)
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.test_loader):
                X, Y = batch
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X)
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
                
                if "cuda" in self.device : torch.cuda.empty_cache()
                
            test_acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            test_loss = np.average(loss_trace)
            print(f'client:{self.id} | Test Acc:{test_acc*100:.2f} | Test Loss:{test_loss:.4f}')
            self.model.to('cpu')

class Server():
    def __init__(self, DM_dict:dict, algorithm:str=None):
        self.train_DM = DM_dict['train']
        self.test_DM = DM_dict['test']
        
        self.clients = None
        self.device = cfg.DEVICE
        
        self.global_model = Net()
        
        self.criterion = nn.CrossEntropyLoss()              # TODO: utils.get_loss(cfg['loss']:str)
        
        self.Algorithm = FedAvg.FedAvg                      # FedAVG 같은 aggrrgation method 들어감 TODO: utils.get_algortihm() 작성
        
        self.mp_flag = False

    def setup(self):
        self.clients = self.create_clients()
        self.data = FEMNIST(self.test_DM.global_test_data)
        self.dataloader = DataLoader(self.data, batch_size=256, shuffle=False)
        self.transmit_model()
        self.setup_clients()
        
        
    def create_clients(self, n_users:int=35):
        self.user_ids = self.test_DM.users
        self.user_ids = np.random.choice(self.user_ids, n_users, replace=False)
        clients = {}
        for user in self.user_ids:
            data_info = {'train':self.train_DM.data[user],\
                         'test':self.test_DM.data[user]}
            clients[user] = Client(client_id=user, model=self.global_model, data_info=data_info, device=self.device)
            clients[user].model = copy.deepcopy(self.global_model)
        return clients
    
    def setup_clients(self)->None:
        for k, client in tqdm(enumerate(self.clients), leave=False):
            self.clients[client].setup()
    
    def transmit_model(self, sampled_clients:list=None)->None:
        if sampled_clients == None:
            for client in tqdm(self.clients, leave=False):
                self.clients[client].model.load_state_dict(copy.deepcopy(self.global_model.state_dict()))
                # self.clients[client].model = copy.deepcopy(self.global_model)
        else:
            for client in tqdm(sampled_clients, leave=False):
                self.clients[client].model.load_state_dict(copy.deepcopy(self.global_model.state_dict()))
                # self.clients[client].model = copy.deepcopy(self.global_model)

        
    def sample_clients(self, n_participant:int=10)->np.array:
        assert n_participant <= len(self.user_ids), "Check 'n_participant <= len(self.clients)'"
        return np.random.choice(self.user_ids, n_participant, replace=False) # 입력된 수의 유저를 추출해서 반환

    def train_selected_clients(self, sampled_clients:list)->None:
        total_sample = 0
        for client in tqdm(sampled_clients, leave=False):
            self.clients[client].local_train()
            total_sample += len(self.clients[client])
        return total_sample
    
    def mp_train_selected_clients(self, procnum:int, client:str)->None:
        self.clients[client].local_train()
    
    def test_selected_models(self, sampled_clients):
        for client in sampled_clients:
            self.clients[client].local_test()

    def mp_test_selected_models(self, procnum:int, client:str):
        self.clients[client].local_test()
    
    def average_model(self, sampled_clients, coefficients):
        averaged_weights = OrderedDict()
        for it, client in tqdm(enumerate(sampled_clients), leave=False):
            local_weights = self.clients[client].model.state_dict()
            for key in self.global_model.state_dict().keys():
                if it == 0:
                    averaged_weights[key] = coefficients[it] * local_weights[key]
                else:
                    averaged_weights[key] += coefficients[it] * local_weights[key]
        self.global_model.load_state_dict(averaged_weights)
    
    def update_model(self, train_result:dict, layers:list=None):
        self.received_models, num_samples = [], []
        for result in train_result:
            self.received_models.append(result['model'])
            num_samples.append(result['num_sample'])
        state = self.Algorithm(self.received_models, num_samples, layers)
        self.global_model.load_state_dict(state)

    def train_federated_model(self):
        sampled_clients = self.sample_clients()
        print(f"CLIENTS {sampled_clients} ARE SELECTED!\n")
        
        if self.mp_flag:
            print("TRAIN WITH MP!\n")
            procs = []
            selected_total_size = []
            for idx, c in enumerate(sampled_clients):
                selected_total_size.append(len(self.clients[c]))
                proc = Process(target=self.mp_train_selected_clients, args=(idx, c))
                proc.start()
                procs.append(proc)
            for proc in procs:
                proc.join()
            selected_total_size = sum(selected_total_size)
            # with pool.ThreadPool(processes=cpu_count() - 1) as workhorse:
            #     selected_total_size = workhorse.map(self.mp_train_selected_clients, sampled_clients)
            
        else:
            print("TRAIN WITH SP!\n")
            selected_total_size = self.train_selected_clients(sampled_clients)

        # if not self.mp_flag:
        #     print("TEST WITH MP!\n")
        #     procs = []
        #     for idx, c in enumerate(sampled_clients):
        #         proc = Process(target=self.mp_test_selected_models, args=(idx, c))
        #         proc.start()
        #         procs.append(proc)
        #     for proc in procs:
        #         proc.join()
        #     # with pool.ThreadPool(processes=cpu_count() - 1) as workhorse:
        #     #     workhorse.map(self.mp_test_selected_models, sampled_clients)
        # else:
        print("TEST WITH SP!\n")
        self.test_selected_models(sampled_clients)
                
        mixing_coefficients = [len(self.clients[client]) / selected_total_size for client in sampled_clients]
        
        # print(f"mixing_coefficients:{mixing_coefficients}")
        # print(f'CLIENT {sampled_clients[0]} WEIGHT')
        # print(f'CLIENT {self.clients[sampled_clients[0]].model.state_dict()["fc1.weight"]} WEIGHT')
        # print(f'CLIENT {sampled_clients[1]} WEIGHT')
        # print(f'CLIENT {self.clients[sampled_clients[1]].model.state_dict()["fc1.weight"]} WEIGHT')
        # print(f'Server Weight before update')
        # print(f'CLIENT {self.global_model.state_dict()["fc1.weight"]} WEIGHT')
        self.average_model(sampled_clients, mixing_coefficients)
        # print(f'Server Weight after update')
        # print(f'CLIENT {self.global_model.state_dict()["fc1.weight"]} WEIGHT')
        self.transmit_model()
        
    def global_test(self):
        self.global_model.eval()
        self.global_model.to(self.device)
        
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.dataloader):
                X, Y = batch
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.global_model(X)
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
            self.acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            self.test_loss = np.average(loss_trace)
            print(f'Global Test Result | Acc:{self.acc*100:.2f}, Loss:{self.test_loss:.4f}')
            self.global_model.to('cpu')
    def fit(self):
        pass

In [4]:
import torch.multiprocessing as mp
from multiprocessing import Process

import torch.autograd as autograd

autograd.set_detect_anomaly(True)

mp.set_start_method('spawn')
PATH = cfg.DATAPATH['femnist']
    
file_dict = get_files(PATH)
    
TRAIN_DM = DataManager(file_dict['train'], is_train=True)
TEST_DM = DataManager(file_dict['test'], is_train=False)
DM_dict = {'train':TRAIN_DM,
           'test':TEST_DM}
print("DATA READY")
    
print(f"WORKING WITH {cfg.DEVICE}")
    

DATA READY
WORKING WITH cuda


In [61]:
server = Server(DM_dict)
print("SERVER READY")
    
server.setup()
print('===== ROUND 0 =====\nServer Setup Complete!')

SERVER READY


===== ROUND 0 =====
Server Setup Complete!


In [62]:
for i in range(10):
    print(f'===== ROUND {i+1} START! =====\n')
    server.train_federated_model()
    server.global_test()
        

  0%|          | 0/10 [00:00<?, ?it/s]

===== ROUND 1 START! =====

CLIENTS ['17' '25' '32' '16' '4' '7' '31' '1' '9' '26'] ARE SELECTED!

TRAIN WITH SP!



 10%|█         | 1/10 [00:13<02:02, 13.62s/it]

=== Client 17 Finished Training 3008 samples ===
client:17 | Train Acc:22.87 | Train Loss:3.2651


 20%|██        | 2/10 [00:42<02:59, 22.45s/it]

...

 90%|█████████ | 9/10 [02:25<00:15, 15.43s/it]

=== Client 33 Finished Training 3037 samples ===
client:33 | Train Acc:77.38 | Train Loss:0.7628


=== Client 5 Finished Training 4751 samples ===
client:5 | Train Acc:78.91 | Train Loss:0.7091
TEST WITH SP!

client:13 | Test Acc:26.53 | Test Loss:4.3846
client:32 | Test Acc:77.46 | Test Loss:0.8324
client:34 | Test Acc:72.46 | Test Loss:0.9960
client:8 | Test Acc:72.76 | Test Loss:1.0034
client:31 | Test Acc:76.55 | Test Loss:0.9238
client:10 | Test Acc:70.66 | Test Loss:1.0772
client:29 | Test Acc:73.87 | Test Loss:0.9328
client:17 | Test Acc:67.44 | Test Loss:1.2062


client:33 | Test Acc:75.00 | Test Loss:0.9275
client:5 | Test Acc:75.79 | Test Loss:0.9208


Global Test Result | Acc:67.20, Loss:1.1167


In [49]:
server.global_model.state_dict()['fc1.weight']

tensor([[-0.0308,  0.0339,  0.0247,  ..., -0.0260,  0.0069,  0.0245],
        [ 0.0048, -0.0336,  0.0300,  ...,  0.0076, -0.0006,  0.0174],
        [ 0.0171, -0.0087, -0.0146,  ...,  0.0111,  0.0015, -0.0210],
        ...,
        [ 0.0073,  0.0097, -0.0242,  ..., -0.0315, -0.0354,  0.0192],
        [-0.0113, -0.0231,  0.0196,  ..., -0.0013, -0.0307,  0.0232],
        [ 0.0082, -0.0187, -0.0180,  ..., -0.0172, -0.0092, -0.0107]])

In [50]:
server.global_test()

Global Test Result | Acc:40.58, Loss:2.4326


In [52]:
server.clients['27'].local_test()

client:27 | Test Acc:49.37 | Test Loss:1.9029


In [53]:
server.clients['27'].local_train()

=== Client 27 Finished Training 6294 samples ===
client:27 | Train Acc:62.42 | Train Loss:1.3118


In [54]:
server.clients['27'].local_test()

client:27 | Test Acc:60.93 | Test Loss:1.3687


In [55]:
server.global_model.state_dict()['fc1.weight']

tensor([[-0.0308,  0.0339,  0.0247,  ..., -0.0260,  0.0069,  0.0245],
        [ 0.0048, -0.0336,  0.0300,  ...,  0.0076, -0.0006,  0.0174],
        [ 0.0171, -0.0087, -0.0146,  ...,  0.0111,  0.0015, -0.0210],
        ...,
        [ 0.0073,  0.0097, -0.0242,  ..., -0.0315, -0.0354,  0.0192],
        [-0.0113, -0.0231,  0.0196,  ..., -0.0013, -0.0307,  0.0232],
        [ 0.0082, -0.0187, -0.0180,  ..., -0.0172, -0.0092, -0.0107]])

In [56]:
server.global_test()

Global Test Result | Acc:40.58, Loss:2.4326


In [59]:
server.clients['27'].optimizer.param_groups[0]['lr']

0.01

# CLIENT

In [2]:
from torch.optim import SGD

class Client():
    def __init__(self, client_id:str, model:nn.Module, data_info:dict=None, device:str=cfg.DEVICE):
        self.id = client_id
        #self.cfg = cfg
        
        self.__model = None
        self.device = device
        
        self.train_info, self.test_info = data_info['train'], data_info['test'] # 함수화하기
        self.trainset, self.testset = FEMNIST(self.train_info), FEMNIST(self.test_info)
        
    @property
    def model(self):             
        return self.__model

    @model.setter
    def model(self, model):
        self.__model = model
    
    def __len__(self):
        return len(self.trainset)
    
    def setup(self):
        self.train_loader = DataLoader(self.trainset, batch_size=16, shuffle=True)
        self.test_loader = DataLoader(self.testset, batch_size=16, shuffle=False)
        self.optimizer = SGD(self.model.parameters(), lr=0.01)     # TODO: utils.get_optimizer(cfg['optim']:str)
        self.criterion = nn.CrossEntropyLoss()                       # TODO: utils.get_loss(cfg['loss']:str)
        self.epochs = 10
    
    def local_train(self)->None:
        proc = os.getpid()
        self.model.train()
        self.model.to(self.device)
        # TRAINING
        for epoch in range(self.epochs):
            for idx, batch in enumerate(self.train_loader):
                self.optimizer.zero_grad()
                X, Y = batch
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X)
                loss = self.criterion(pred, Y)
                loss.backward()
                self.optimizer.step()
                if "cuda" in self.device : torch.cuda.empty_cache()
        # TESTING
        self.model.eval()
        self.model.to(self.device)
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.train_loader):
                X, Y = batch
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X)
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
                if "cuda" in self.device : torch.cuda.empty_cache()
            train_acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            train_loss = np.average(loss_trace)
            self.model.to('cpu')
        
        print(f'=== Process ID: {proc} | Client {self.id} Finished Training {len(self)} samples ===')
        print(f'client:{self.id} | Train Acc:{train_acc*100:.2f} | Train Loss:{train_loss:.4f}')
    
    def local_test(self):
        self.model.eval()
        self.model.to(self.device)
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.test_loader):
                X, Y = batch
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.model(X)
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
                
                if "cuda" in self.device : torch.cuda.empty_cache()
                
            test_acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            test_loss = np.average(loss_trace)
            print(f'client:{self.id} | Test Acc:{test_acc*100:.2f} | Test Loss:{test_loss:.4f}')
            self.model.to('cpu')

# SERVER

In [3]:
class Server():
    def __init__(self, DM_dict:dict, algorithm:str=None):
        self.train_DM = DM_dict['train']
        self.test_DM = DM_dict['test']
        
        self.clients = None
        self.device = cfg.DEVICE
        
        self.global_model = Net()
        
        self.criterion = nn.CrossEntropyLoss()              # TODO: utils.get_loss(cfg['loss']:str)
        
        self.Algorithm = FedAvg.FedAvg                      # FedAVG 같은 aggrrgation method 들어감 TODO: utils.get_algortihm() 작성
        self.received_models = None                         # Client.upload_model() 결과가 여기 들어감
        
        self.mp_flag = True

    def setup(self):
        self.clients = self.create_clients()
        self.data = FEMNIST(self.test_DM.get_global_testset())
        self.dataloader = DataLoader(self.data, batch_size=256, shuffle=False)
        
        self.transmit_model()
        self.setup_clients()
        
        
    def create_clients(self, n_users:int=100):
        self.user_ids = self.test_DM.users
        self.user_ids = np.random.choice(self.user_ids, n_users, replace=False)
        clients = {}
        for user in self.user_ids:
            data_info = {'train':self.train_DM.get_user_info(user),\
                         'test':self.test_DM.get_user_info(user)}
            clients[user] = Client(client_id=user, model=self.global_model, data_info=data_info)
        return clients
    
    def setup_clients(self)->None:
        for k, client in tqdm(enumerate(self.clients), leave=False):
            self.clients[client].setup()
    
    def transmit_model(self, sampled_clients:list=None)->None:
        if sampled_clients == None:
            for client in tqdm(self.clients, leave=False):
                self.clients[client].model = copy.deepcopy(self.global_model)
        else:
            for client in tqdm(sampled_clients, leave=False):
                self.clients[client].model = copy.deepcopy(self.global_model)

        
    def sample_clients(self, n_participant:int=50)->np.array:
        assert n_participant <= len(self.user_ids), "Check 'n_participant <= len(self.clients)'"
        return np.random.choice(self.user_ids, n_participant, replace=False) # 입력된 수의 유저를 추출해서 반환

    def train_selected_clients(self, sampled_clients:list)->None:
        total_sample = 0
        for client in tqdm(sampled_clients, leave=False):
            self.clients[client].local_train()
            total_sample += len(self.clients[client])

    def mp_train_selected_clients(self, client:str)->None:
        self.clients[client].local_train()
        n_sample = len(self.clients[client])
        return n_sample
    
    def test_selected_models(self, sampled_clients):
        for client in sampled_clients:
            self.clients[client].local_test()

    def mp_test_selected_models(self, client):
        self.clients[client].local_test()
    
    def average_model(self, sampled_clients, coefficients):
        averaged_weights = OrderedDict()
        for it, client in tqdm(enumerate(sampled_clients), leave=False):
            local_weights = self.clients[client].model.state_dict()
            for key in self.global_model.state_dict().keys():
                if it == 0:
                    averaged_weights[key] = coefficients[it] * local_weights[key]
                else:
                    averaged_weights[key] += coefficients[it] * local_weights[key]
        self.global_model.load_state_dict(averaged_weights)
    
    def update_model(self, train_result:dict, layers:list=None):
        self.received_models, num_samples = [], []
        for result in train_result:
            self.received_models.append(result['model'])
            num_samples.append(result['num_sample'])
        state = self.Algorithm(self.received_models, num_samples, layers)
        self.global_model.load_state_dict(state)

    def train_federated_model(self):
        sampled_clients = self.sample_clients()
        
        if self.mp_flag:
            with pool.ThreadPool(processes=cpu_count() - 1) as workhorse:
                selected_total_size = workhorse.map(self.mp_train_selected_clients, sampled_clients)
            selected_total_size = sum(selected_total_size)
        else:
            selected_total_size = self.train_selected_clients(sampled_clients)

        if self.mp_flag:
            with pool.ThreadPool(processes=cpu_count() - 1) as workhorse:
                workhorse.map(self.mp_test_selected_models, sampled_clients)
        else:
            self.test_selected_models(sampled_clients)
        
        mixing_coefficients = [len(self.clients[client]) / selected_total_size for client in sampled_clients]
        
        self.average_model(sampled_clients, mixing_coefficients)
        
    def global_test(self):
        self.global_model.eval()
        self.global_model.to(self.device)
        
        with torch.no_grad():
            loss_trace, result_pred, result_anno = [], [], []
            for idx, batch in enumerate(self.dataloader):
                X, Y = batch
                X, Y = X.to(self.device), Y.to(self.device)
                pred = self.global_model(X)
                loss = self.criterion(pred, Y)
                loss_trace.append(loss.to('cpu').detach().numpy())
                pred_np  = pred.to('cpu').detach().numpy()
                pred_np  = np.argmax(pred_np, axis=1).squeeze()
                Y_np     = Y.to('cpu').detach().numpy().reshape(-1, 1).squeeze()
                result_pred = np.hstack((result_pred, pred_np))
                result_anno = np.hstack((result_anno, Y_np))
            self.acc = metrics.accuracy_score(y_true=result_anno, y_pred=result_pred)
            self.test_loss = np.average(loss_trace)
            print(f'Global Test Result | Acc:{self.acc*100:.2f}, Loss:{self.test_loss:.4f}')
            self.global_model.to(self.device)

In [4]:
PATH = cfg.DATAPATH['femnist']
    
file_dict = get_files(PATH)
    
TRAIN_DM = DataManager(file_dict['train'], is_train=True)
TEST_DM = DataManager(file_dict['test'], is_train=False)
DM_dict = {'train':TRAIN_DM,
            'test':TEST_DM}
print("DATA READY")

DATA READY


In [ ]:
server = Server(DM_dict)
#server.create_clients()
server.setup()
for i in range(100):
    server.train_federated_model()
    server.global_test()


In [ ]:
file = server.train_DM.files[0]
with open(file) as f:
    data = json.load(f)
data


In [41]:
np.shape(data['user_data'][data['users'][0]]['x'])

(141, 784)

In [67]:
class DataManager():
    def __init__(self, files:list, is_train:bool=True):
        self.files = files
        self.is_train = is_train
        self.users, self.data = [], {}
        if not self.is_train:
            self.global_test_data = {'x':[], 'y':[]}
        
        for idx, file in enumerate(self.files):
            idx = str(idx)
            self.data[idx] = {'x':[], 'y':[]}
            with open(file) as f:
                data = json.load(f)
                self.users.append(idx)

                for user in data['users']:              # 각 유저의 data 저장
                    self.data[idx]['x'] = self.data[idx]['x'] + data['user_data'][user]['x']
                    self.data[idx]['y'] = self.data[idx]['y'] + data['user_data'][user]['y']
                    
                if not self.is_train:                   # test dataset인 경우 global evaluation 위해서 모든 데이터셋 저장
                    for user in data['users']:
                        self.global_test_data['x'] = self.global_test_data['x'] + data['user_data'][user]['x']
                        self.global_test_data['y'] = self.global_test_data['y'] + data['user_data'][user]['y']

class FEMNIST(Dataset):
    def __init__(self, data:dict):
        self.data = data
        self.data['x'] = np.array(data['x'])
        self.data['y'] = np.array(data['y'])
        
    def __getitem__(self, idx):
        #self.X = torch.tensor(self.data['x'][idx,:].reshape(1, 28, 28)).float()
        self.X = torch.tensor(self.data['x'][idx,:]).float()
        self.Y = torch.tensor(self.data['y'][idx]).long()
        return self.X, self.Y

    def __len__(self):
        return len(self.data['y'])

In [53]:
PATH = '../leaf/data/femnist/data/train'
files = [os.path.join(PATH, file) for file in os.listdir(PATH) if file.endswith('.json')]
files.sort()

DM = DataManager(files, is_train=False)

In [49]:
np.shape(DM.data['1']['x'])

(351, 784)

In [66]:
np.shape(DM.data['0']['x'])

(2754, 784)

In [68]:
dataset = FEMNIST(DM.data['0'])

In [69]:
dataset[0]

(tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
         1.0000, 1.0000, 1.0